In [ ]:
!pip install tqdm

In [ ]:
import base64
import os
import pandas as pd
import requests
import time
import json
from tqdm import tqdm
import math
from google.colab import userdata
from openai import OpenAI
# OpenAI API Key
api_key = userdata.get('api_key')#need to add the key to the 'secrets section'
client = OpenAI(api_key = api_key)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Implementation Evaluation

**The prompt for this phase's evaluation:**
> Hello Networking Intent Implementation Evaluator, I added here {file_number} files: '{input_configuration_file_name}' which is the initial network component configuration file, '{input_textual_topology_name}' which is the initial topology description, an intent file named 'intent.txt', and {len(files)} updated files which shows the changes after intent implementation named {files_names}.
Here are the scoring keys for your evaluation:
{scoring_keys}
Given the these files, and the scoring keys, assign a grade to the intent implementation according to your instructions, and briefly explain your scoring for each key.,


In [ ]:
# ### assistant creation
# temp = 0.0
# selected_model = 'gpt-4.1-mini'

# client.beta.assistants.create(
#         model= selected_model,
#         name = "Intent Implementation Evaluator Assistant",
#         description="A Networking Intent Implementation Evaluator",
#         instructions = 'You are a helpful networking intent implementation evaluator assistant. Your role as an evaluator is to assess the accuracy of network intent implementation by grading it. You will review the provided files:
# 1. Initial Network Components Configuration – The starting configuration of the network devices.
# 2. Initial Topology Description – The original structure of the network.
# 3. Textual Intent – A detailed description of the desired changes or outcomes for the network's configuration.
# 4. Updated Network Components Configuration files or topology files – Several files which show the changes made to the topology or to the configuration based on the intent. The number of those files can be one or more.

# Evaluation Process:
# 1. Understand the Intent: Carefully analyze the textual intent to fully grasp the desired changes or outcomes.
# 2. Analyze Initial Files: Examine the initial network configuration and topology to understand the starting state and context.
# 3. Evaluate Updated Files: Compare the updated configurations against the intent to determine if the desired changes have been accurately implemented.

# Scoring and Grading:
# - Scoring Keys: You will be provided with specific scoring keys by the user to guide your evaluation. Each of those keys specifies a condition for noncompliance with the intent implementation, and the point deductions for this specific noncompliance.
# 	- Points should only be deducted if the updated file complies with the key’s condition (i.e., if the issue described in the key is present in the implementation).
# 	- Partial deductions are allowed for partial compliance or minor issues related to a key.
# - Grading:
# 	- Start with a grade of 100.
# 	- If the implementation meets the noncompliance criteria described in a key, subtract the corresponding points from the grade.
# 	- Ensure deductions align strictly with the provided scoring keys and do not penalize issues outside their scope.

# Response Guidelines:
# - Provide the final grade as a numeric value (0–100).
# - Adhere strictly to these instructions and the provided scoring keys during the evaluation process.
# - In your response, respond **only** with a single JSON object that conforms to the following schema:
#  ```json
#     {
#         "explanation":<your reasoning process and explanations alongside corresponding point deductions>,
#         "final_grade": <numerical grade>,

#     }
#     ```',
#         tools=[{"type": "file_search"}],
#         temperature = temp,
#         response_format={
#             "type": "json_schema",
#             "json_schema": {
#                 "name": "evaluator_response",
#                 "schema":{
#                     "type": "object",
#                     "properties":{
#                         "explanation": {"type": "string"},
#                         "final_grade": {"type": "number", "minimum": 0,"maximum": 100},
#                     },
#                     "required": ["explanation", "final_grade"],
#                     "additionalProperties": False
#                 },
#                 "strict": True
#             },
#         }
#     )



In [ ]:
#----------- Run Related Values----------------
eval_assistant_temp = 0
model = "gpt-4.1-mini"
run_number = 1
#----------------------------------------------------------------

In [ ]:
def create_eval_thread(intent, input_config, input_topology, input_topology_name, updated_files_path, scoring_keys_address):

  input_configuration_file_name = 'Total_Configs.txt'
  input_textual_topology_name = f'{input_topology_name}.json'

  files = [f for f in os.listdir(updated_files_path) if os.path.isfile(os.path.join(updated_files_path, f))]

  updated_files = []
  for file in files:
    file_path = os.path.join(updated_files_path, file)
    assistant_file = client.files.create(file=open(file_path, "rb"), purpose='assistants')
    updated_files.append((file, assistant_file.id))
  input_config_file = client.files.create(file=open(input_config, "rb"), purpose='assistants')
  input_topology_file = client.files.create(file=open(input_topology, "rb"), purpose='assistants')
  intent_file = client.files.create(file=open(intent, "rb"), purpose='assistants')

  with open( scoring_keys_address, 'r') as file:
    scoring_keys = file.read()

  attachments = [
        {"file_id": input_config_file.id, "tools": [{"type": "file_search"}]},
        {"file_id": input_topology_file.id, "tools": [{"type": "file_search"}]},
        {"file_id": intent_file.id, "tools": [{"type": "file_search"}]}]

  for file in updated_files:
    attachments.append({"file_id": file[1], "tools": [{"type": "file_search"}]})

  file_number = 3+len(files)

  if len(files) == 1:
    updated_file_message = f"and an updated file which shows the changes after intent implementation named {updated_files[0][0]}"
  else:
    files_names = ""
    for i in range(len(files)-1):
      files_names = files_names+f"{updated_files[i][0]}, "
    files_names = files_names+f" and {updated_files[-1][0]}"
    updated_file_message = f"and {len(files)} updated files which shows the changes after intent implementation named {files_names}"


  message_thread = client.beta.threads.create(
    messages=[
      {
        "role": "user",
        "content": f"""Hello Networking Intent Implementation Evaluator, I added here {file_number} files: '{input_configuration_file_name}' which is the initial network component configuration file, '{input_textual_topology_name}' which is the initial topology description, an intent file named 'intent.txt', {updated_file_message}.
Here are the scoring keys for your evaluation:
{scoring_keys}
Given the these files, and the scoring keys, assign a grade to the intent implementation according to your instructions, and briefly explain your scoring for each key.""",
        #----------------- ADDITIONAL LINES FOR NEW API CODE-----------------------------
        "attachments": attachments,
        #-----------------------------------------------------------------------------------
      },
    ]
  )
  return message_thread

In [ ]:
folders =["Adding_DMZ", "Adding_DRA", "Adding_Local_PCs","Internet_Connectivity", "Adding_Communication_Servers","Role_Based_CLI_Access","Time_Based_Access_List", "Transparent_IOS_Firewall", "Basic_Zone_Based_Firewall","IP_Traffic_Export"]
input_config_directory = "path"
updated_specs_directory = f"path/{model}"
scoring_keys_directory = "path"

results_folder = f"path/{model}"

os.makedirs(results_folder, exist_ok=True)
if not os.path.isfile(f"/path/{model}/Run{run_number}_tracking.csv"):
  itterations_file = pd.read_csv("path/Eval_result_template.csv")
else:
  itterations_file = pd.read_csv(f"path/{model}/Run{run_number}_tracking.csv")

for index, row in tqdm(itterations_file.iterrows(), total=len(itterations_file)):
  if row["Phase4_Eval"]=="V": #done
    continue

  platform = row["Platform"]
  diagram_type = row["Diagram_Type"]
  scenario = row["Scenario"]
  # skip if result files do not exist


  if not os.path.exists(f"path/Implementation_results/{model}/{platform}/{diagram_type}/(Run{run_number})/{scenario}/final_results"):
    print(f"path/{model}/{platform}/{diagram_type}/(Run{run_number})/{scenario}/final_results")
    continue

  output_dir_address = os.path.join(results_folder, platform, diagram_type,f"(run{run_number})")
  os.makedirs(output_dir_address, exist_ok=True)
  output_file_address = os.path.join(output_dir_address, f"{scenario}_results.txt")

  output_file = open(output_file_address, "a")

  input_config_scenario_folder = os.path.join(input_config_directory, scenario)
  input_config_file_address = os.path.join(input_config_scenario_folder,"Total_Configs.txt")
  intent_file_address = os.path.join(input_config_scenario_folder,"intent.txt")
  updated_specs_scenario_directory = os.path.join(updated_specs_directory, platform, diagram_type,f"(Run{run_number})", scenario)
  scoring_keys_address = os.path.join(scoring_keys_directory, f"{scenario}.txt")

  updated_spec_run_directory = os.path.join(updated_specs_scenario_directory,"final_results")

   ## different ground truth representations - only four types are needed to cover the 9 combinations)
  if platform == "PowerPoint":
     if diagram_type == "No_Labels_On_Edges": #(pp: no_lables)
       gt_file_path = f"path/No_Labels_On_Edges/{scenario}.json"
     else: #(pp: normal,messy)
       gt_file_path = f"path/PowerPoint/Normal/{scenario}.json"
  else:
    if diagram_type == "No_Labels_On_Edges": #(no_labels:gns,paper)
      gt_file_path = f"path/No_Labels_On_Edges/{scenario}.json"
    else: #(normal: gns,paper, messy: gns,paper)
      gt_file_path = f"path/Normal/{scenario}.json"



  message_thread = create_eval_thread(intent_file_address, input_config_file_address, gt_file_path, scenario, updated_spec_run_directory, scoring_keys_address)
  thread_id = message_thread.id
  # run the thread
  run = client.beta.threads.runs.create_and_poll(
    temperature = eval_assistant_temp,
    thread_id = thread_id,
    assistant_id = assist_num)

  print(f"\n-----------[PLATFORM: {platform}, DIAGRAM TYPE: {diagram_type}, SCENARIO: {scenario}]--------------\n")
  # output_file.write(f"\n--------- RUN:{run_number}]-------------------\n")
  if run.status == 'completed':
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for message in messages.data:
      if message.role == "assistant":
        messg_id = message.id
        value = message.content[0].text.value
        output_file.write(f"{value}\n")
        print(value)
  output_file.close()

  0%|          | 0/90 [00:00<?, ?it/s]/tmp/ipython-input-1858307746.py:47: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message_thread = client.beta.threads.create(
/tmp/ipython-input-276507296.py:82: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create_and_poll(
/tmp/ipython-input-276507296.py:90: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(thread_id=thread_id)
100%|██████████| 90/90 [00:19<00:00,  4.63it/s]


-----------[PLATFORM: Paper_Sketches, DIAGRAM TYPE: Normal, SCENARIO: IP_Traffic_Export]--------------

{"explanation":"1. The next hop of router Ditto was set correctly in the route-map EXPORT_TO_IDS with 'set ip next-hop 192.168.24.4' for both HTTP and EIGRP traffic, so no deduction here.\n\n2. The interfaces on Ditto are correctly defined with IP addresses matching the initial configuration: FastEthernet0/0 (192.168.12.2), FastEthernet1/0 (192.168.23.2), and FastEthernet2/0 (192.168.24.2). IDS interface FastEthernet0/0 is also correctly configured with 192.168.24.4. No interfaces are missing or wrongly defined, so no deduction.\n\n3. The ACL named HTTP_TRAFFIC on Ditto permits TCP traffic from hosts 192.168.12.1 and 192.168.23.3 to 192.168.24.4 on port 80 (HTTP). The route-map EXPORT_TO_IDS matches this ACL and also matches protocol eigrp, setting the next hop to 192.168.24.4. This correctly allows EIGRP and HTTP traffic to be exported to IDS. No issues found here.\n\n4. The route-

## Extracting eval grades


In [ ]:
results_folder = f"path/{model}"
os.makedirs(results_folder, exist_ok=True)
if not os.path.isfile(f"path/{model}/Run{run_number}_tracking.csv"):
  itterations_file = pd.read_csv("path/Eval_result_template.csv")
else:
  itterations_file = pd.read_csv(f"path/{model}/Run{run_number}_tracking.csv")

for index, row in itterations_file.iterrows():
  if row["Phase4_Eval"] == "V": #done
    continue

  platform = row["Platform"]
  diagram_type = row["Diagram_Type"]
  scenario = row["Scenario"]

  result_dir_address = os.path.join(results_folder, platform, diagram_type,f"(run{run_number})",f"{scenario}_results.txt")
  if not os.path.exists(result_dir_address):
    continue
  with open(result_dir_address, "r") as f:
    data_str = f.read()
  try:
    result = json.loads(data_str)
    row['Phase4_Eval'] = 'V'
    row['Phase4 Grade'] = result['final_grade']
  except:
    row['Phase4_Eval'] = 'X'
  itterations_file.iloc[index] = row
itterations_file.to_csv(f"path/{model}/Run{run_number}_tracking.csv", index=False)
